# Figure panel 2

In [ ]:
options(warn=-1)

In [ ]:
library_load <- suppressMessages(
    
    suppressWarnings(
        
        list(
        
            # Seurat 
            library(Seurat), 

            # TradeSeq
            library(tradeSeq),

            # Data 
            library(tidyverse), 
            library(data.table), 

            # miloR
            library(miloR), 
            library(ggbeeswarm), 

            # Plotting 
            library(ggplot2), 
            library(patchwork), 
            library(cowplot), 
            library(ComplexHeatmap), 

            # Pyhton compatibility
            library(reticulate)

        )
    )
)

In [ ]:
# Configure reticulate 
# use_condaenv(condaenv="p.3.10.16-FD20200109SPLENO", conda="/nobackup/peer/fdeckert/miniconda3/bin/conda", required=NULL)
# py_config()

In [ ]:
random_seed <- 42
set.seed(random_seed)

In [ ]:
# Set working directory to project root
setwd("/research/peer/fdeckert/FD20200109SPLENO")

In [ ]:
# Source
source("bin/so_pl.R")
source("bin/tradeseq_pp.R")
source("bin/tradeseq_pl.R")
source("bin/gsea_pl.R")
source("bin/gsea_pp.R")

In [ ]:
# Plotting Theme
source("plotting_global.R")
ggplot2::theme_set(theme_global_set(size_select=4)) # From project global source()

# Load reference data 

In [ ]:
tf <- read.table("/research/peer/fdeckert/reference/animaltfdb/Mus_musculus_TF.txt")[[2]]

lr <- CellChat::CellChatDB.mouse[[1]] %>% dplyr::select(pathway_name, ligand, receptor.symbol, receptor.family, annotation) %>% dplyr::filter(annotation %in% c("Cell-Cell Contact", "Secreted Signaling", "ECM-Receptor")) %>% 
    separate_rows(receptor.symbol, sep=", ") %>% dplyr::distinct() %>%
    separate_rows(ligand, sep=", ") %>% dplyr::distinct() %>% 
    dplyr::rename(pathway=pathway_name, receptor=receptor.symbol, family=receptor.family)

# Import scRNAseq data 

In [ ]:
so <- readRDS("data/scRNAseq/object/pp_0.rds")
so <- NormalizeData(so)

In [ ]:
so$sample_group <- factor(so$sample_group, levels=c("Bl6_NaCl_D6", "Bl6_CpG_D6"))

# UMAP erytime WT

In [ ]:
dpt <- read.csv("result/pseudotime/ery_dpt.csv", row.names=1)
so <- AddMetaData(so, dpt)

In [ ]:
fp_1 <- fplot(so, features="dpt_pseudotime", pt_size=0.25, option="F", size_select=4)
fp_1[[1]]$data <- fp_1[[1]]$data[!is.na(fp_1[[1]]$data$features), ]

In [ ]:
pdf("result/figures/1_scRNAseq/panel_2/umap_erytime.pdf", width=3.5, height=2.5)

gridExtra::grid.arrange(
    
    fp_1 %>% egg::set_panel_size(., width=unit(4.5, "cm"), height=unit(4.5, "cm"))

)

dev.off()

# Heatmap of smoothed expression 

In [ ]:
tradeseq_res <- readRDS("result/lineage/tradeseq_res_1.rds")
fitgam <- tradeseq_res[["fitgam"]]
ptpg <- tradeseq_res[["ptpg"]]

In [ ]:
contrast_vec <- c("Bl6_NaCl_D6", "Bl6_CpG_D6")
ptpg <- ptpg[ptpg$contrast==paste0(contrast_vec, collapse=":"), ]

In [ ]:
genes_ptcg <- ptpg[ptpg$ptpg_class %in% c("Canonical"), ][["gene"]]
genes_ptpg <- ptpg[!ptpg$ptpg_class %in% c("Canonical"), ][["gene"]]

In [ ]:
color_mat <- color$sample_group[contrast_vec]

## PTCG Ctl and D6

In [491]:
pt_split_1 <- pt_split(fitgam, genes_ptcg, contrast_vec)

In [492]:
row_genes <- c(

    "Meis1", "Klf1", "Tal1", "Gata1", "Gata2", # Transcription factors 
    "Mki67", "Top2a", # Proliferation 
    "Pkm", # Glycolysis
    "Pdk1", # Glycolysis/aerobic metabolism
    "Hif1a", "Ldha", # Glycolysis/anaerboic metabolism
    "Atp5a1", "Atp5b", # Oxidative phosphorylation 
    "Sod2", # ROS detoxifiyer
    "G6pd", "Pgd", # PP
    "Cpt1a", "Acadl", # b-oxidatse
    "Mtor", "Rptor", "Rheb",  # mTOR pathwy
    "Tfam", "Ssbp1", "Polg", # Mitochondrial maintainance
    "Bnip3l", # Mitochondrial removal 
    "Dnmt1", "Dnmt3b", "Tet2", "Tet3", # Epigenetics
    "Alas2", "Ncoa4", # Heme metabolism 
    "Icam4", "Epor", "Tfrc" # Receptor

)

row_genes %in% names(pt_split_1)
row_genes <- row_genes[row_genes %in% names(pt_split_1)]

row_anno <- rowAnnotation(
    
    labels=anno_mark(
        
        at=which(names(pt_split_1) %in% row_genes),
        labels=names(pt_split_1)[which(names(pt_split_1) %in% row_genes)],
        side="left", 
        labels_gp=gpar(fontsize=6), 
        link_width=unit(3, "mm")
    
    )

)

[1]  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE
[13]  TRUE  TRUE FALSE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE
[25]  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE

In [493]:
pt_hm_1 <- pt_hm(fitgam, names(pt_split_1), "Bl6_NaCl_D6", contrast_vec, 50, color_mat[1], row_split=pt_split_1, width=0.75, height=180, fontsize_select=2, use_raster=TRUE)
pt_hm_2 <- pt_hm(fitgam, names(pt_split_1), "Bl6_CpG_D6", contrast_vec, 50, color_mat[2], row_split=pt_split_1, width=0.75, height=180, fontsize_select=2, use_raster=TRUE)

In [494]:
pdf("result/figures/1_scRNAseq/panel_2/hm_ptcg_ctl_d6.pdf", width=3, height=10)

draw(row_anno + pt_hm_1 + pt_hm_2)

dev.off()

pdf 
  2

## PTPG D6 DOWN vs Ctl

In [ ]:
pt_split_1 <- pt_split(fitgam, ptpg[ptpg$ptpg_class %in% c("Bl6_NaCl_D6"), ]$gene, "Bl6_NaCl_D6", n_points=101)

In [ ]:
row_genes <- c(
    
    "Lyl1", "Ifi27", "Usp18", "Oasl1", "Slc3a2", "Slc44a1", "Slc20a1", "Slc25a24"

)

row_genes %in% names(pt_split_1)
row_genes <- row_genes[row_genes %in% names(pt_split_1)]

row_anno <- rowAnnotation(
    
    labels=anno_mark(
        
        at=which(names(pt_split_1) %in% row_genes),
        labels=names(pt_split_1)[which(names(pt_split_1) %in% row_genes)],
        side="left", 
        labels_gp=gpar(fontsize=6), 
        link_width=unit(3, "mm")
    
    )

)

In [ ]:
pt_hm_1 <- pt_hm(fitgam, names(pt_split_1), "Bl6_NaCl_D6", contrast_vec, 50, color_mat[1], row_split=pt_split_1, width=0.75, height=100, fontsize_select=2, use_raster=TRUE)
pt_hm_2 <- pt_hm(fitgam, names(pt_split_1), "Bl6_CpG_D6", contrast_vec, 50, color_mat[2], row_split=pt_split_1, width=0.75, height=100, fontsize_select=2, use_raster=TRUE)

In [ ]:
pdf("result/figures/1_scRNAseq/panel_2/hm_ptpg_d6_down.pdf", width=3, height=3)

draw(row_anno + pt_hm_1 + pt_hm_2)

dev.off()

## PTPG D6 UP vs Ctl

In [ ]:
pt_split_1 <- pt_split(fitgam, ptpg[ptpg$ptpg_class %in% "Bl6_CpG_D6", ]$gene, "Bl6_CpG_D6", n_points=101)

In [ ]:
row_genes <- c(

    "Stat1", "Sox6",
    "Gbp2", "Gbp7", "Iigp1", "Ifi47", "Isg20", "Apol11b", # Interferon  
    "Ccnb1", "Cdc20", "Ube2c", # Cell cycle
    "App", "Hbb-bt", "Hba-a2"

)

row_genes %in% names(pt_split_1)
row_genes <- row_genes[row_genes %in% names(pt_split_1)]

row_anno <- rowAnnotation(
    
    labels=anno_mark(
        
        at=which(names(pt_split_1) %in% row_genes),
        labels=names(pt_split_1)[which(names(pt_split_1) %in% row_genes)],
        side="left", 
        labels_gp=gpar(fontsize=6), 
        link_width=unit(3, "mm")
    
    )

)

In [ ]:
pt_hm_1 <- pt_hm(fitgam, names(pt_split_1), "Bl6_NaCl_D6", contrast_vec, 50, color_mat[1], row_split=pt_split_1, width=0.75, height=100, fontsize_select=2, use_raster=TRUE)
pt_hm_2 <- pt_hm(fitgam, names(pt_split_1), "Bl6_CpG_D6", contrast_vec, 50, color_mat[2], row_split=pt_split_1, width=0.75, height=100, fontsize_select=2, use_raster=TRUE)

In [ ]:
pdf("result/figures/1_scRNAseq/panel_2/hm_ptpg_d6_up.pdf", width=3, height=3)

draw(row_anno + pt_hm_1 + pt_hm_2)

dev.off()

# Density celltype plot 

In [ ]:
dp_1 <- ggplot(so[, so$sample_group %in% contrast_vec[1] & !is.na(so$dpt_pseudotime)]@meta.data, aes(x=na.omit(dpt_pseudotime), y=after_stat(density), fill=celltype_low, alpha=1)) + 
    geom_density(adjust=3.0, size=unit(0.5/2.141959, "pt")) + 
    guides(fill=guide_legend(nrow=2), alpha="none") + 
    scale_fill_manual(values=color$celltype_low) + 
    scale_y_continuous(breaks=c(0, 4, 8), labels=c("0", "4", "8")) + 
    scale_x_continuous(breaks=c(0.0, 0.5, 1.0), labels=c("0.0", "0.5", "1.0"), limits=c(0, 1), expand=c(0, 0)) + 
    theme(legend.position="none")

In [ ]:
pdf("result/figures/1_scRNAseq/panel_2/denp_dpt_ctl.pdf", width=1.5, height=6)

gridExtra::grid.arrange(
    
    dp_1 %>% egg::set_panel_size(., width=unit(18.75, "mm"), height=unit(0.75, "cm")), 
    dp_1 %>% egg::set_panel_size(., width=unit(20, "mm"), height=unit(0.75, "cm")), nrow=2
    
)

dev.off()

In [ ]:
dp_1 <- ggplot(so[, so$sample_group %in% contrast_vec[2] & !is.na(so$dpt_pseudotime)]@meta.data, aes(x=na.omit(dpt_pseudotime), y=after_stat(density), fill=celltype_low, alpha=1)) + 
    geom_density(adjust=3.0, size=unit(0.5/2.141959, "pt")) + 
    guides(fill=guide_legend(nrow=2), alpha="none") + 
    scale_fill_manual(values=color$celltype_low) + 
    scale_y_continuous(breaks=c(0, 5, 10), labels=c("0", "5", "10")) + 
    scale_x_continuous(breaks=c(0.0, 0.5, 1.0), labels=c("0.0", "0.5", "1.0"), limits=c(0, 1), expand=c(0, 0)) + 
    theme(legend.position="none")

In [ ]:
pdf("result/figures/1_scRNAseq/panel_2/denp_dpt_d6.pdf", width=1.5, height=6)

gridExtra::grid.arrange(
    
    dp_1 %>% egg::set_panel_size(., width=unit(18.75, "mm"), height=unit(0.75, "cm")), 
    dp_1 %>% egg::set_panel_size(., width=unit(20, "mm"), height=unit(0.75, "cm")), nrow=2
    
)

dev.off()

# Lineplot pseudotime expression 

## PTCG Ctl and D6

In [487]:
options(repr.plot.width=5, repr.plot.height=5)

genes <- c("Meis1", "Itga2b", "Vim", "Fes", "Ncoa4", "Kit", "Gata2", "Gata1", "Klf1", "Mki67", "Epor", "Tfrc", "Gypa", "Alas2", "Tet2", "Tet3", "Ncoa4")
genes %in% genes_ptcg

[1] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[16] TRUE TRUE

In [488]:
lp_1 <- lapply(genes, function(i) {plot_smooth(fitgam, gene=i, point=FALSE, condition_color=color_mat, line_size=unit(1.5/2.141959, "pt")) + annotate("text", x=0.5, y=0, label=i, hjust=0.5, vjust=-1, size=2) + theme(legend.position="none")})
lp_1 <- lapply(lp_1, function(p) egg::set_panel_size(p, width=unit(2.0, "cm"), height=unit(0.75, "cm")))
lp_1 <- do.call(gridExtra::arrangeGrob, c(lp_1, ncol=1, nrow=ceiling(length(lp_1)/1)))

In [489]:
pdf("result/figures/1_scRNAseq/panel_2/lp_ptcg_ctl_and_d6.pdf", width=1*1.25, height=ceiling(length(lp_1)/1))

grid::grid.draw(lp_1)

dev.off()

pdf 
  2

## PTPG D6 vs Ctl

In [ ]:
genes <- ptpg[ptpg$ptpg_class=="Perturbed", ]$gene
genes <- pt_split(fitgam, genes, contrast_vec) %>% names()

In [ ]:
lp_1 <- lapply(genes, function(i) {plot_smooth(fitgam, gene=i, point=FALSE, condition_color=color_mat, line_size=unit(1.5/2.141959, "pt")) + annotate("text", x=0.5, y=0, label=i, hjust=0.5, vjust=-1, size=2) + theme(legend.position="none")})
lp_1 <- lapply(lp_1, function(p) egg::set_panel_size(p, width=unit(2.0, "cm"), height=unit(0.75, "cm")))
lp_1 <- do.call(gridExtra::arrangeGrob, c(lp_1, ncol=10, nrow=ceiling(length(lp_1)/10)))

In [ ]:
pdf("result/figures/1_scRNAseq/panel_2/lp_ptpg_ctl_vs_d6.pdf", width=10*1.25, height=ceiling(length(lp_1)/10))

grid::grid.draw(lp_1)

dev.off()

## PTPG D6 DOWN vs Ctl

In [495]:
genes <- ptpg[ptpg$ptpg_class=="Bl6_NaCl_D6", ]$gene
genes <- pt_split(fitgam, genes, contrast_vec) %>% names()

In [496]:
c("Rps3a1", "Txnip", "Akr1b3", "Cox6b2", "Pdcd4", "Car1", "Prss50", "Cela1") %in% genes

[1] FALSE FALSE FALSE FALSE FALSE FALSE FALSE  TRUE

In [ ]:
lp_1 <- lapply(genes, function(i) {plot_smooth(fitgam, gene=i, point=FALSE, condition_color=color_mat, line_size=unit(1.5/2.141959, "pt")) + annotate("text", x=0.5, y=0, label=i, hjust=0.5, vjust=-1, size=2) + theme(legend.position="none")})
lp_1 <- lapply(lp_1, function(p) egg::set_panel_size(p, width=unit(2.0, "cm"), height=unit(0.75, "cm")))
lp_1 <- do.call(gridExtra::arrangeGrob, c(lp_1, ncol=10, nrow=ceiling(length(lp_1)/10)))

In [ ]:
pdf("result/figures/1_scRNAseq/panel_2/lp_ptpg_d6_down_vs_ctl.pdf", width=10*1.25, height=ceiling(length(lp_1)/10))

grid::grid.draw(lp_1)

dev.off()

## PTPG D6 UP vs Clt

In [ ]:
genes <- ptpg[ptpg$ptpg_class=="Bl6_CpG_D6", ]$gene
genes <- pt_split(fitgam, genes, contrast_vec) %>% names()

In [ ]:
lp_1 <- lapply(genes, function(i) {plot_smooth(fitgam, gene=i, point=FALSE, condition_color=color_mat, line_size=unit(1.5/2.141959, "pt")) + annotate("text", x=0.5, y=0, label=i, hjust=0.5, vjust=-1, size=2) + theme(legend.position="none")})
lp_1 <- lapply(lp_1, function(p) egg::set_panel_size(p, width=unit(2.0, "cm"), height=unit(0.75, "cm")))
lp_1 <- do.call(gridExtra::arrangeGrob, c(lp_1, ncol=10, nrow=ceiling(length(lp_1)/10)))

In [ ]:
pdf("result/figures/1_scRNAseq/panel_2/lp_ptpg_d6_up_vs_ctl.pdf", width=10*1.25, height=ceiling(length(lp_1)/10))

grid::grid.draw(lp_1)

dev.off()

# Pie chart PT gene counts

In [ ]:
ptpg_class_color <- c("Perturbed"="#7f7f7f", "Canonical"="#AEAEAE")
data <- table(ptpg %>% dplyr::mutate(ptpg_class=ifelse(ptpg_class=="Canonical", "Canonical", "Perturbed")) %>% dplyr::pull(ptpg_class)) %>% data.frame() %>% dplyr::rename(group=Var1, count=Freq) %>% dplyr::mutate(group=factor(group, levels=names(ptpg_class_color)))
data
p_pl_1 <- ggplot(data, aes(x="", y=count, fill=group)) +
    geom_col(width=1) +
    coord_polar(theta="y") + 
    scale_fill_manual(values=ptpg_class_color) + 
    ggtitle("Ctl (wt) vs D6 (wt)") + 
    guides(fill=guide_legend(ncol=1, override.aes=list(alpha=1, size=1.5), keywidth=0.25, keyheight=0.25, default.unit="cm"))

In [ ]:
ptpg_class_color <- color$sample_group[c("Bl6_NaCl_D6", "Bl6_CpG_D6")]
data <- table(ptpg %>% dplyr::filter(ptpg_class %in% names(ptpg_class_color)) %>% dplyr::pull(ptpg_class)) %>% data.frame() %>% dplyr::rename(group=Var1, count=Freq) %>% dplyr::mutate(group=factor(group, levels=names(ptpg_class_color)))
data
p_pl_2 <- ggplot(data, aes(x="", y=count, fill=group)) +
    geom_col(width=1) +
    coord_polar(theta="y") + 
    scale_fill_manual(values=ptpg_class_color) + 
    ggtitle("Ctl (+/+) vs D1 (+/+)") + 
    guides(fill=guide_legend(ncol=1, override.aes=list(alpha=1, size=1.5), keywidth=0.25, keyheight=0.25, default.unit="cm"))

In [ ]:
pdf("result/figures/1_scRNAseq/panel_2/p_pl_ptg_counts.pdf", width=5, height=2.5)

gridExtra::grid.arrange(
    
    p_pl_1 %>% egg::set_panel_size(., width=unit(2.5, "cm"), height=unit(2.5, "cm")),
    p_pl_2 %>% egg::set_panel_size(., width=unit(2.5, "cm"), height=unit(2.5, "cm")),
    ncol=2
    
)

dev.off()